In [5]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import numpy as np
from typing import Dict, Any, Optional, Tuple
import json
import joblib

from src.wandb_utils import WandbLogger
from src.metrics import compute_metrics
from src.models import KmerSequenceModel
from src.metrics import compute_metrics, compute_regression_metrics

from scipy import sparse
import os

from sklearn.decomposition import TruncatedSVD
import pandas as pd
from src.utils import load_kmer_counts, load_svd
from src.data import DNASequenceDataset
from src.data import KmerTokenizer

from sklearn.metrics import roc_auc_score, average_precision_score
from torcheval.metrics import MulticlassAUROC, MulticlassAUPRC, MulticlassAccuracy


def svm_project_kmer_vec(svd_model, full_dataset, batch_size=128):
    projected_vecs = []
    n_samples = full_dataset.shape[0]   # number of rows
    for i in range(0, n_samples, batch_size):
        batch = full_dataset[i:i+batch_size]
        projected_vecs.append(batch @ svd_model.components_.T)
    return np.vstack(projected_vecs)

#Run twice to generate sparse kmer counts for 1kb and 5kb
def generate_vec_dataset(resolution: str, svd_dim: int, batch_size: int=128):
    data_path = f"../data/data_transformation/chipseq/updated_histone_mods_and_GA_factors.feature_matrix.{resolution}_res.txt"
    df = pd.read_csv(data_path, sep="\t")
    df['seq'] = df['seq'].str.upper()
        
    tokenizer = KmerTokenizer(k=8)
    df['kmer_vec'] = df['seq'].map(lambda x: tokenizer.seq_to_vec(x))
        
    full_dataset = sparse.vstack(df['kmer_vec'])
    sparse.save_npz(f"../data/sparse_kmer_counts_{resolution}.npz", full_dataset)
    
#Creates and saves final datasets from sparse kmer counts
def build_dataset(resolution: str, svd_dim: int, batch_size: int=128):
    data_path = f"../data/data_transformation/chipseq/updated_histone_mods_and_GA_factors.feature_matrix.{resolution}_res.txt"
    df = pd.read_csv(data_path, sep="\t")
    df['seq'] = df['seq'].str.upper()
    
    sparse_counts_path = f"../data/sparse_kmer_counts_{resolution}.npz"
    full_dataset = load_kmer_counts(sparse_counts_path, drop_unk=True)
    
    svd_path = f"../output/models/seq_svm/svd_model_{resolution}_{svd_dim}.joblib"
    if not os.path.exists(svd_path):
        svd_model = TruncatedSVD(n_components=svd_dim)
        svd_model.fit(full_dataset)
        joblib.dump(svd_model, svd_path)
        
    svd_model = load_svd(n_components=svd_dim, path=svd_path)
    projected_vecs = svm_project_kmer_vec(svd_model, full_dataset, batch_size)
    
    #DNA Sequence dataset from projection
    sequences = df['seq']
    gene_labels = df['gene_labels'].to_numpy()
    mre_labels = df['mre_labels'].to_numpy()
    metadata = df.drop(columns=['seq', 'gene_labels', 'mre_labels'])
    
    final_dataset = DNASequenceDataset(sequences, gene_labels, mre_labels, projected_vecs, metadata)
    
    torch.save(final_dataset, f"../data/{resolution}_{svd_dim}_vec_dataset.pt")
    return final_dataset




# TODO: @Romer, feel free to port the notebook code into this format to make it easier...
class SeqSVMTrainer:
    def __init__(self,
                 dataset: DNASequenceDataset,
                 resolution: str,
                 svd_dim: int,
                 input_dim: int = 128, 
                 hidden_dims: Tuple[int, ...] = (256, 128),
                 num_classes: int = 3, 
                 dropout: float = 0.3,
                 batch_size: int = 128, 
                 lr: float = 1e-3, 
                 weighted: bool = False,
                 task: str = "mre",
                 target_metric: str = "MulticlassAccuracy",
                 wandb_logger: Optional[WandbLogger] = None,
                 device: Optional[torch.device] = None):
               
        self.device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.wandb_logger = wandb_logger
        self.batch_size = batch_size
        self.weighted = weighted
        self.task = task
        self.target_metric = target_metric
        self.resolution = resolution
        self.svd_dim = svd_dim
        
        
        self.run_name = f"{task}_res{resolution}_svd{svd_dim}_lr{lr}_bs{batch_size}_weighted{weighted}"
               
        #Making training and validation data loaders
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        self.train_dataset, self.val_dataset = random_split(dataset, [train_size, val_size])
        self.train_loader = DataLoader(self.train_dataset, batch_size=batch_size, shuffle=True)
        self.val_loader = DataLoader(self.val_dataset, batch_size=batch_size, shuffle=False)

        #Instantiating the model
        self.model = KmerSequenceModel(
            input_dim=input_dim,
            hidden_dims=hidden_dims,
            num_classes=num_classes,
            dropout=dropout,
        ).to(self.device)    

        if weighted:
            # Compute class weights
            if task == "mre":
                _, class_counts = np.unique(dataset.mre_labels, return_counts=True)
            else:
                _, class_counts = np.unique(dataset.gene_labels, return_counts=True)
            weights = 1 / (class_counts / class_counts.sum())
            weights = torch.tensor(weights, dtype=torch.float32).to(self.device)
            self.criterion = nn.CrossEntropyLoss(weight=weights)
        else:
            self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
               

    def train(self, val_metrics=None, epochs: int=10):
        if val_metrics is None:
            val_metrics = [
            MulticlassAUROC(num_classes=3),
            MulticlassAUPRC(num_classes=3),
            MulticlassAccuracy(),
        ]
        best_state_dict = None
        
        device = self.device
        patience = 0
        best_metric = 0
        train_acc = MulticlassAccuracy().to(device)
        best_metrics = {}
        
        for epoch in range(epochs):
            if patience >= 5:
                print(f"Early Stopping at epoch {epoch}")
                break
            self.model.train()
            train_loss = 0.0
            
            for gene, mre, vecs in self.train_loader:
                if self.task == "mre":
                    labels = mre
                elif self.task == "gene":
                    labels = gene
                labels = labels.long().to(self.device)
                vecs = vecs.to(self.device)
                
                self.optimizer.zero_grad()
                outs = self.model(vecs)
                loss = self.criterion(outs, labels)
                loss.backward()
                self.optimizer.step()
                
                logits = F.softmax(outs, dim=1)
                train_loss += loss.detach().item()
                train_acc.update(logits, labels)
                
            train_accuracy = train_acc.compute().item()
            print(f"[Epoch {epoch}] Train Loss: {train_loss:.4f}, Acc: {train_accuracy:.4f}")
            train_acc.reset()
                
            val_loss, metrics = self.evaluate(val_metrics=val_metrics)
            print(f"[Epoch {epoch}] Val Loss: {val_loss:.4f}, Val Metrics: {metrics}")
            
            
            wandb.log({
                "epoch": epoch,
                "train_loss": train_loss,
                "train_accuracy": train_accuracy,
                "val_loss": val_loss,
                **{f"val_{k}": v for k, v in metrics.items()}
            })
            
#             if self.wandb_logger:
#                 self.wandb_logger.log({
#                     "epoch": epoch,
#                     "train_loss": train_loss,
#                     "train_accuracy": train_accuracy,
#                     "val_loss": val_loss,
#                     **{f"val_{k}": v for k, v in metrics.items()}
#                 })



                
            target_metric = metrics.get(self.target_metric, 0.0)
            if target_metric > best_metric:
                patience = 0
                best_metric = target_metric
                best_metrics = metrics
                #torch.save(self.model.state_dict(), f"{self.run_name}_best.pt")
                best_state_dict = self.model.state_dict()
            else:
                patience +=1
                
        #best_model = torch.load(f"{self.run_name}_best.pt", map_location=device)
        #self.model.load_state_dict(best_model)
        
        if best_state_dict is not None:
            torch.save(best_state_dict, f"{self.run_name}_best.pt")
            self.model.load_state_dict(best_state_dict)

        
        
        self.model.to(device)

        return self.model, best_metrics
               

    def evaluate(self, val_metrics=None, return_full: bool=False):
        device = self.device
        self.model.eval()
        
        for m in val_metrics:
            m.to(device)
            m.reset()
        val_loss = 0
        lab_pred_log = ([], [], [])
        
        with torch.no_grad():
            for gene, mre, vecs in self.val_loader:
                labels = mre if self.task == "mre" else gene
                labels = labels.long().to(device)
                vecs = vecs.to(device)
                
                outs = self.model(vecs)
                loss = self.criterion(outs, labels)
                val_loss += loss.detach().item()
                logits = F.softmax(outs, dim=1)
                preds = torch.argmax(logits, dim=1)
                
                if return_full:
                    for append, ls in zip([labels, preds, logits], lab_pred_log):
                        ls.append(append.detach().cpu())
                    
                for metric in val_metrics:
                    metric.update(logits, labels)
        
        lab_pred_log = [torch.cat(ls) for ls in lab_pred_log] if return_full else None
        metrics = {m.__class__.__name__: m.compute().item() for m in val_metrics}

        return val_loss, metrics if not return_full else (val_loss, metrics, lab_pred_log)
      

    def input_gradients(self):
        device = self.device
        self.model.eval()
        all_loss_grads = []
        all_class_grads = []
        
        for gene, mre, vecs in self.val_loader:
            labels = mre if self.task == "mre" else gene
            labels = labels.long().to(device)

            vecs = vecs.to(device)
            vecs.requires_grad_(True)

            outputs = self.model(vecs)

            # per-class gradients: d(outputs[:,c].sum())/d(vecs)
            class_grads = []
            C = outputs.shape[1]
            for c in range(C):
                grad_c = torch.autograd.grad(
                    outputs[:, c].sum(), vecs, retain_graph=True
                )[0]
                class_grads.append(grad_c.detach().cpu())
            class_grads = torch.stack(class_grads, dim=1)  # (B, C, D)

            loss = self.criterion(outputs, labels)
            loss_grads = torch.autograd.grad(loss, vecs)[0].detach().cpu()

            all_loss_grads.append(loss_grads)
            all_class_grads.append(class_grads)

        loss_grads = torch.cat(all_loss_grads, dim=0)   # (N, D)
        class_grads = torch.cat(all_class_grads, dim=0) # (N, C, D)
        return loss_grads, class_grads
        


In [2]:
import wandb
wandb.login()

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: romer_miranda (bind-gps) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [6]:
def train_sweep():
    wandb.init()
    config = wandb.config
    
    wandb.run.name = f"{config.task}_{config.resolution}_{config.svd_dim}_{config.lr}"
    
    dataset_path = f"../data/{config.resolution}_{config.svd_dim}_vec_dataset.pt"
    dataset = torch.load(dataset_path, weights_only=False)
    dataset.vecs = torch.tensor(dataset.vecs, dtype=torch.float32)


    #initialize trainer 
    trainer = SeqSVMTrainer(
        dataset=dataset,
        resolution=config.resolution,
        svd_dim=config.svd_dim,
        input_dim=config.svd_dim,
        hidden_dims=config.hidden_dims,
        batch_size=config.batch_size,
        lr=config.lr,
        weighted=config.weighted,
        task=config.task,
        target_metric=config.target_metric,
        wandb_logger=WandbLogger(), 
        device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    )

    val_metrics = [
        MulticlassAUROC(num_classes=3),
        MulticlassAUPRC(num_classes=3),
        MulticlassAccuracy(),
    ]
    trainer.train(val_metrics=val_metrics, epochs=config.epochs)
    


sweep_config = {
    "method": "grid",
    "metric": {
        "name": "val_MulticlassAUROC", 
        "goal": "maximize"
    },
    "parameters": {
        "resolution": {"values": ["1kb", "5kb"]},
        "svd_dim": {"values": [64, 128, 256]},
        "hidden_dims": {"values": [(256, 128), (128, 64)]},
        "batch_size": {"values": [128]},
        "lr": {"values": [1e-3, 1e-4, 1e-5]},
        "weighted": {"values": [True, False]},
        "task": {"values": ["mre", "gene"]},
        "target_metric": {"values": ["MulticlassAUROC", "MulticlassAccuracy", "MulticlassAUPRC"]},
        "epochs": {"values": [10]}
    }
}



sweep_id = wandb.sweep(sweep=sweep_config, project="BindGPS")
wandb.agent(sweep_id, function=train_sweep, count=100)
wandb.finish()

Create sweep with ID: 79ufxypb
Sweep URL: https://wandb.ai/bind-gps/BindGPS/sweeps/79ufxypb


wandb: Agent Starting Run: ru4xx79v with config:
wandb: 	batch_size: 128
wandb: 	epochs: 10
wandb: 	hidden_dims: [256, 128]
wandb: 	lr: 0.001
wandb: 	resolution: 1kb
wandb: 	svd_dim: 64
wandb: 	target_metric: MulticlassAUROC
wandb: 	task: mre
wandb: 	weighted: True
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: 
wandb: 🚀 View run hopeful-sweep-5 at: https://wandb.ai/bind-gps/BindGPS/runs/9z2fasxr
wandb: Find logs at: wandb/run-20251002_121429-9z2fasxr/logs


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


[Epoch 0] Train Loss: 801.4778, Acc: 0.6766
[Epoch 0] Val Loss: 185.2773, Val Metrics: {'MulticlassAUROC': 0.8065464496612549, 'MulticlassAUPRC': 0.5224514603614807, 'MulticlassAccuracy': 0.6978011727333069}
[Epoch 1] Train Loss: 752.6460, Acc: 0.7245
[Epoch 1] Val Loss: 184.0574, Val Metrics: {'MulticlassAUROC': 0.8172297477722168, 'MulticlassAUPRC': 0.5284170508384705, 'MulticlassAccuracy': 0.7183718085289001}
[Epoch 2] Train Loss: 736.1022, Acc: 0.7278
[Epoch 2] Val Loss: 180.8344, Val Metrics: {'MulticlassAUROC': 0.822375476360321, 'MulticlassAUPRC': 0.5315660238265991, 'MulticlassAccuracy': 0.723132848739624}
[Epoch 3] Train Loss: 729.2272, Acc: 0.7320
[Epoch 3] Val Loss: 180.5910, Val Metrics: {'MulticlassAUROC': 0.8148261904716492, 'MulticlassAUPRC': 0.5280207395553589, 'MulticlassAccuracy': 0.732763946056366}
[Epoch 4] Train Loss: 724.3335, Acc: 0.7372
[Epoch 4] Val Loss: 181.0924, Val Metrics: {'MulticlassAUROC': 0.8298683166503906, 'MulticlassAUPRC': 0.537189245223999, 'Multi

...

epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▅▆▆▇▇▇███
train_loss,█▅▄▃▃▂▂▂▁▁
val_MulticlassAUPRC,▁▄▅▆▆▇▇▇▇█
val_MulticlassAUROC,▁▄▅▆▆▇▇▇██
val_MulticlassAccuracy,▁▃▅▅▅▇▇▅▇█
val_loss,█▅▄▃▃▂▂▂▁▁
epoch,9
train_accuracy,0.55784
train_loss,796.61639
val_MulticlassAUPRC,0.53441


wandb: Agent Starting Run: i4qjfrcu with config:
wandb: 	batch_size: 128
wandb: 	epochs: 10
wandb: 	hidden_dims: [256, 128]
wandb: 	lr: 0.0001
wandb: 	resolution: 1kb
wandb: 	svd_dim: 256
wandb: 	target_metric: MulticlassAUROC
wandb: 	task: gene
wandb: 	weighted: False
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


[Epoch 0] Train Loss: 778.8718, Acc: 0.5758
[Epoch 0] Val Loss: 178.0251, Val Metrics: {'MulticlassAUROC': 0.7124812006950378, 'MulticlassAUPRC': 0.5022011995315552, 'MulticlassAccuracy': 0.6315100789070129}
[Epoch 1] Train Loss: 715.2503, Acc: 0.6185
[Epoch 1] Val Loss: 174.6831, Val Metrics: {'MulticlassAUROC': 0.7241435647010803, 'MulticlassAUPRC': 0.5194044709205627, 'MulticlassAccuracy': 0.6372160911560059}
[Epoch 2] Train Loss: 703.6936, Acc: 0.6263
[Epoch 2] Val Loss: 172.3660, Val Metrics: {'MulticlassAUROC': 0.7302940487861633, 'MulticlassAUPRC': 0.528144121170044, 'MulticlassAccuracy': 0.6414319276809692}
[Epoch 3] Train Loss: 695.2714, Acc: 0.6313
[Epoch 3] Val Loss: 171.6104, Val Metrics: {'MulticlassAUROC': 0.7333877086639404, 'MulticlassAUPRC': 0.5316388010978699, 'MulticlassAccuracy': 0.6446301937103271}
[Epoch 4] Train Loss: 687.8997, Acc: 0.6367
[Epoch 4] Val Loss: 170.4420, Val Metrics: {'MulticlassAUROC': 0.737145721912384, 'MulticlassAUPRC': 0.5354375839233398, 'Mul

epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▅▆▆▇▇▇███
train_loss,█▄▃▃▂▂▂▁▁▁
val_MulticlassAUPRC,▁▄▅▅▆▆▇▇▇█
val_MulticlassAUROC,▁▃▅▅▆▆▇▇▇█
val_MulticlassAccuracy,▁▃▅▆▆▆▇▇▆█
val_loss,█▆▄▄▃▃▂▂▃▁
epoch,9
train_accuracy,0.64811
train_loss,668.79909
val_MulticlassAUPRC,0.549


In [4]:
"""Ran once to generate datasets for each config of resolution + num components for svd.
No need to run again"""

# resolutions = ["1kb", "5kb"]
# svd_dims = [64, 128, 256]
# batch_size = 128 

# # going over each possible config
# for res in resolutions:
#     sparse_path = f"../data/sparse_kmer_counts_{res}.npz"
    
#     if os.path.exists(sparse_path):
#         print(f"Sparse kmer dataset already exists for resolution {res}, skipping generation.")
#     else:
#         print(f"Generating sparse kmer dataset for resolution: {res}")
#         generate_vec_dataset(resolution=res, svd_dim=max(svd_dims), batch_size=batch_size)
    
    
#     for svd_dim in svd_dims:
#         print(f"Building final dataset for resolution: {res}, SVD dim: {svd_dim}")
#         final_dataset = build_dataset(resolution=res, svd_dim=svd_dim, batch_size=batch_size)
#         print(f"Saved dataset: ../data/{res}_{svd_dim}_vec_dataset.pt")


Sparse kmer dataset already exists for resolution 1kb, skipping generation.
Building final dataset for resolution: 1kb, SVD dim: 64
Saved dataset: ../data/1kb_64_vec_dataset.pt
Building final dataset for resolution: 1kb, SVD dim: 128
Saved dataset: ../data/1kb_128_vec_dataset.pt
Building final dataset for resolution: 1kb, SVD dim: 256
Saved dataset: ../data/1kb_256_vec_dataset.pt
Generating sparse kmer dataset for resolution: 5kb
Building final dataset for resolution: 5kb, SVD dim: 64
Saved dataset: ../data/5kb_64_vec_dataset.pt
Building final dataset for resolution: 5kb, SVD dim: 128
Saved dataset: ../data/5kb_128_vec_dataset.pt
Building final dataset for resolution: 5kb, SVD dim: 256
Saved dataset: ../data/5kb_256_vec_dataset.pt


In [4]:
import torch

path = "../data/1kb_64_vec_dataset.pt"  # pick one of your saved files
dataset = torch.load(path, weights_only=False)


print(type(dataset))
for i in range(5):  # first 5 samples
    sample = dataset[i]
    print(f"Sample {i}:")
    print(sample)
    print("---")

#According to /src/data.py, each sample returns: (gene label), (mre label), (projected vecs)

<class 'src.data.DNASequenceDataset'>
Sample 0:
(np.float64(0.0), np.float64(1.0), array([ 4.29903625e+00, -5.07841464e-02, -5.95490218e-02,  2.23849635e-02,
       -1.95563769e+00, -2.74832220e-01, -1.47271345e-01, -5.98321708e-01,
       -1.01756960e-01,  3.75994648e-01, -2.96419350e-01,  1.39606044e-01,
       -8.28622317e-02,  1.15172188e+00, -1.14388889e+00,  1.88835120e-01,
       -3.49444820e-01, -1.43968937e-01,  1.13821193e-01, -1.03384529e-03,
        1.07563701e+00,  3.05132409e+00,  7.52321911e-02,  3.85161554e-02,
       -3.28594366e-01,  1.53618512e+00,  1.69692086e+00, -2.87948883e-01,
       -3.59926167e-01,  1.64228357e+00,  1.43106586e+00,  8.42300228e+00,
        4.35640748e+00, -9.48458329e+00, -6.89165702e+00,  9.71942104e+00,
        6.20992529e+00,  2.62728328e+01, -3.33701884e+00, -1.09555262e+01,
       -4.38510073e+00,  1.13880819e+01,  6.86413200e+00, -1.17881584e+01,
       -9.31975395e+00,  1.38754807e+00, -1.05953968e+01,  2.16327412e+00,
        5.6521908